In [11]:
import comet_ml
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
import datasets
from transformers import logging, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from itertools import combinations
import numpy as np
import api_keys
import re

In [3]:
experiment = comet_ml.Experiment(api_key=api_keys.comet, project_name='rap-lyrics-generator-llm')

SyntaxError: invalid syntax (3153767716.py, line 1)

In [ ]:
experiment.log_parameters({
    "num_train_epochs": 3,
    "per_device_train_batch_size": 2,
    # Add any other relevant hyperparameters here
})

In [10]:
rap_lyrics_train_dataset = datasets.load_dataset("nateraw/rap-lyrics-v2", split='train')
rap_lyrics_train_dataset = rap_lyrics_train_dataset[:int(len(rap_lyrics_train_dataset)*0.7)]
rap_lyrics_test_dataset = datasets.load_dataset("nateraw/rap-lyrics-v2", split='train')[int(len(rap_lyrics_train_dataset)*0.7):]

In [ ]:
model_name = "gpt2"  # You could use a larger model like gpt2-medium for better performance
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=8092)

# Tokenize dataset
tokenized_train_dataset = rap_lyrics_train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Create a data collator for dynamic batching
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

tokenized_test_dataset = rap_lyrics_train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
data_collator_test = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    report_to="comet_ml",  # Ensure training logs are sent to Comet
)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=None,  # Add compute_metrics function if needed
)

# Fine-tune the model
trainer.train()

In [ ]:
# Save the fine-tuned model
model.save_pretrained('./fine_tuned_rap_model')
tokenizer.save_pretrained('./fine_tuned_rap_model')

In [ ]:
metrics = trainer.evaluate(eval_dataset=tokenized_test_dataset)
experiment.log_metrics(metrics)

In [ ]:
experiment.end()